In [78]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
import os
import random

from skimage import transform, io

from PIL import Image
import cv2

from sklearn.model_selection import train_test_split

In [84]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [85]:
device

'cpu'

# Prepare data

In [2]:
dataset_path = './drive/MyDrive/AIMedic/data/'
dataset_D = './drive/MyDrive/AIMedic/data/Data_D'

### Create CSV file with pandas for paths

In [3]:
data_dictionary = {}

In [ ]:
filepaths = []
filelist = os.listdir(dataset_D)
for dir in filelist:
  print(dir)
  temp_dir = os.path.join(dataset_D, dir)
  for f in os.listdir(temp_dir):
    filepaths.append(os.path.join(temp_dir, f))
  data_dictionary[dir] = filepaths[:1229]
  filepaths = []


In [69]:
covid_path_dataframe = pd.DataFrame(data_dictionary)

In [70]:
covid_path_dataframe

,COVID,non-COVID
0,./drive/MyDrive/AIMedic/data/Data_D/COVID/Covi...,./drive/MyDrive/AIMedic/data/Data_D/non-COVID/...
1,./drive/MyDrive/AIMedic/data/Data_D/COVID/Covi...,./drive/MyDrive/AIMedic/data/Data_D/non-COVID/...
2,./drive/MyDrive/AIMedic/data/Data_D/COVID/Covi...,./drive/MyDrive/AIMedic/data/Data_D/non-COVID/...
3,./drive/MyDrive/AIMedic/data/Data_D/COVID/Covi...,./drive/MyDrive/AIMedic/data/Data_D/non-COVID/...
4,./drive/MyDrive/AIMedic/data/Data_D/COVID/Covi...,./drive/MyDrive/AIMedic/data/Data_D/non-COVID/...
...,...,...
1224,./drive/MyDrive/AIMedic/data/Data_D/COVID/Covi...,./drive/MyDrive/AIMedic/data/Data_D/non-COVID/...
1225,./drive/MyDrive/AIMedic/data/Data_D/COVID/Covi...,./drive/MyDrive/AIMedic/data/Data_D/non-COVID/...
1226,./drive/MyDrive/AIMedic/data/Data_D/COVID/Covi...,./drive/MyDrive/AIMedic/data/Data_D/non-COVID/...
1227,./drive/MyDrive/AIMedic/data/Data_D/COVID/Covi...,./drive/MyDrive/AIMedic/data/Data_D/non-COVID/...


In [65]:
covid_path_dataframe.to_csv('./drive/MyDrive/AIMedic/data/Data_D/data.csv')

> first we want to create dataloader for Dataset_D

In [7]:
covid_path_dataframe

,COVID,non-COVID
0,./drive/MyDrive/AIMedic/data/Data_D/COVID/Covi...,./drive/MyDrive/AIMedic/data/Data_D/non-COVID/...
1,./drive/MyDrive/AIMedic/data/Data_D/COVID/Covi...,./drive/MyDrive/AIMedic/data/Data_D/non-COVID/...
2,./drive/MyDrive/AIMedic/data/Data_D/COVID/Covi...,./drive/MyDrive/AIMedic/data/Data_D/non-COVID/...
3,./drive/MyDrive/AIMedic/data/Data_D/COVID/Covi...,./drive/MyDrive/AIMedic/data/Data_D/non-COVID/...
4,./drive/MyDrive/AIMedic/data/Data_D/COVID/Covi...,./drive/MyDrive/AIMedic/data/Data_D/non-COVID/...
...,...,...
1224,./drive/MyDrive/AIMedic/data/Data_D/COVID/Covi...,./drive/MyDrive/AIMedic/data/Data_D/non-COVID/...
1225,./drive/MyDrive/AIMedic/data/Data_D/COVID/Covi...,./drive/MyDrive/AIMedic/data/Data_D/non-COVID/...
1226,./drive/MyDrive/AIMedic/data/Data_D/COVID/Covi...,./drive/MyDrive/AIMedic/data/Data_D/non-COVID/...
1227,./drive/MyDrive/AIMedic/data/Data_D/COVID/Covi...,./drive/MyDrive/AIMedic/data/Data_D/non-COVID/...


# loader images

In [6]:
csv_path = './drive/MyDrive/AIMedic/data/Data_D/data.csv'
covid_path_dataframe = pd.read_csv(csv_path, index_col=0)

In [17]:
class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        # print(sample.keys())
        image_i = sample['image_i']
        label_i = sample['label_i']
        image_mu = sample['image_mu']
        label_mu = sample['label_mu']

        # swap color axis because
        # numpy image: H x W x C
        # torch image: C x H x W
        image_i = image_i.transpose((2, 0, 1))
        image_mu = image_mu.transpose((2, 0, 1))
        return {'image_i': torch.from_numpy(image_i),
                'label_i': torch.tensor(label_i),
                'image_mu': torch.from_numpy(image_mu),
                'label_mu': torch.tensor(label_mu),
        }

In [89]:
class Rescale(object):
    """Rescale the image in a sample to a given size.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def resize(self, image):

      img = cv2.resize(image, (self.output_size, self.output_size))
      img = np.stack((img,)*3, axis=-1)

      return np.divide(img, 255.0, dtype='float')

    def __call__(self, sample):
        image_i = sample['image_i']
        label_i = sample['label_i']
        image_mu = sample['image_mu']
        label_mu = sample['label_mu']

        image_i = self.resize(image_i)
        image_mu = self.resize(image_mu)

        # h and w are swapped for landmarks because for images,
        # x and y axes are axis 1 and 0 respectively
        # landmarks = landmarks * [new_w / w, new_h / h]

        return {'image_i': image_i,
                'label_i': label_i,
                'image_mu': image_mu,
                'label_mu': label_mu
                }


In [90]:
class CovidDataset(Dataset):
  def __init__(self, path_df, transform=None):
    self.path_df = path_df
    # columns = path_df.columns
    # self.covid_df = path_df[columns[0]]
    # self.nonCovid_df = path_df[columns[1]]

    self.df_list = []
    for column in path_df.columns:
      self.df_list.append(path_df[column])

    self.transform = transform

  def __len__(self):
    return len(self.path_df)

  def __getitem__(self, idx):
    if torch.is_tensor(idx):
      idx = idx.tolist()

    t_i = random.getrandbits(1)
    t_mu = int(not t_i)

    image_i = cv2.imread(self.df_list[t_i].iloc[idx], 0)
    # image_i = np.stack((image_i,)*3, axis=-1)
    label_i = t_i

    image_mu = cv2.imread(self.df_list[t_mu].iloc[idx], 0)
    # image_mu = np.stack((image_mu,)*3, axis=-1)
    label_mu = t_mu

    sample = {
        'image_i': image_i,
        'label_i': label_i,
        'image_mu': image_mu,
        'label_mu': label_mu
    }

    # print('1: ', sample.keys())
    if self.transform:
      # print('2: ', sample.keys())
      sample = self.transform(sample)

    return sample


    

### split dataframe and feed to dataloader

In [91]:
train_df, test_df = train_test_split(covid_path_dataframe, test_size=.2, random_state=1234)

In [92]:
train_dataset = CovidDataset(train_df, 
                     transform=transforms.Compose([
                     Rescale(224),
                     ToTensor(),
                        ])
)
test_dataset = CovidDataset(test_df, 
                     transform=transforms.Compose([
                     Rescale(224),
                     ToTensor(),
                        ])
)

In [93]:
print(f'shape of train dataset: {len(train_dataset)}\nshape of test dataset: {len(test_dataset)}')

shape of train dataset: 983
shape of test dataset: 246


In [94]:
train_loader = DataLoader(train_dataset, batch_size=16, 
                          shuffle=True, num_workers=0)

test_loader = DataLoader(test_dataset, batch_size=16, 
                          shuffle=False, num_workers=0)

# Model

## Mutex Attention Block

In [108]:
class Mutex_block(nn.Module):
    def __init__(self):
        super(Mutex_block, self).__init__()
        # self.batch = batch
        # self.H = H
        # self.C = C
        # self.W = W
        self.softmax = nn.Softmax(dim=1)
        
        
    def forward(self, Frei, Frem):
        batch, C, H, W = Frei.shape
        
        Fam = torch.subtract(Frei, Frem)
        Fam = torch.pow(Fam, 2)
        
        #reshape
        Fam = Fam.view(batch, C, H*W)
        
        #softmax
        Fam = self.softmax(Fam)

        #reshape
        Fam = Fam.view(batch, C, H, W)
        
        #multiplication
        Fam = torch.mul(Fam, Frem)
        
        return Fam

## Fusion Attention Block

In [109]:
class Fusion_block(nn.Module):
    def __init__(self, in_fc, out_fc, pool_size):
        super(Fusion_block, self).__init__()
        self.in_fc = in_fc
        self.out_fc = out_fc
        
        self.softmax = nn.Softmax(dim=1)
        self.avgpool = nn.AvgPool2d(pool_size)
        self.maxpool = nn.MaxPool2d(pool_size)
        
        self.softmax = nn.Softmax(dim=0)
        
        # Set these parameters
        self.fcC = nn.Sequential(
            # nn.Dropout(0.5),
            nn.Linear(in_fc, out_fc),
            nn.ReLU())
        
        self.fcC_prim = nn.Sequential(
            # nn.Dropout(0.5),
            nn.Linear(out_fc, out_fc),
            nn.ReLU())
        
        self.fcM = nn.Sequential(
            # nn.Dropout(0.5),
            nn.Linear(out_fc, out_fc),
            nn.ReLU())
        
        self.fcN = nn.Sequential(
            # nn.Dropout(0.5),
            nn.Linear(out_fc, out_fc),
            nn.ReLU())
        
    def forward(self, Fam, Frem):
        f_temp = torch.add(Fam, Frem)
        # print(f'first add: {f_temp.shape}')
        
        avg_pool = self.avgpool(f_temp).squeeze()
        max_pool = self.maxpool(f_temp).squeeze()
        f_temp = torch.add(avg_pool, max_pool)
        # print(f'add after poolings: {f_temp.shape}')
        
        f_temp = f_temp.view(f_temp.size(0), -1)
        # print(f'bifore first fc: {f_temp.shape}')
        
        
        f_temp = self.fcC(f_temp)
        # print(f'after first fc: {f_temp.shape}')
        f_temp = self.fcC_prim(f_temp)
        # print(f'after second fc: {f_temp.shape}')
        
        
        fM = self.fcM(f_temp)
        # print(f'size of fM: {fM.shape}')
        fN = self.fcN(f_temp)
        # print(f'size of fN: {fN.shape}')
        
        # unsqueeze
        fM = torch.unsqueeze(fM, 0)
        fN = torch.unsqueeze(fN, 0)
        # print(f'unsqueeze fM: {fM.shape}\nunsqueeze fN: {fN.shape}')
        
        # concatenate
        z = torch.concat([fM, fN], dim=0)
        # print(f'size of concat fM, fN: {z.shape}')
        
        # softmax
        z = self.softmax(z)
        # print(f'after softmax: {z.shape}')
        
        fM = z[0,...]
        # print(f'separate fM: {fM.shape}')
        fN = z[1,...]
        # print(f'separate fN: {fN.shape}')
        
        
        # print(f'size of fM after view: {fM.view(fM.size(0), fM.size(1), 1, 1).shape}')
        
        # fM --> [B, C]
        # Fam --> [B, C, H, W]
        Ffm_m = torch.mul(Fam, fM.view(fM.size(0), fM.size(1), 1, 1))
        Ffm_n = torch.mul(Frem, fN.view(fN.size(0), fN.size(1), 1, 1))
        
        Ffm = torch.add(Ffm_m, Ffm_n)
        
        return Ffm

## Final model

### Res Block

In [110]:
class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, 
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes,
                               kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()

        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
              nn.Conv2d(in_planes, self.expansion*planes, 
                        kernel_size=1, stride=stride, bias=False),
              nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


### Final Model

In [111]:
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, Mutex_attention, Fusion_attention, num_classes=2):
        super(ResNet, self).__init__()
        self.in_planes = 64
        
        self.layer0 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(64)
        )
        self.mutex_block_0 = Mutex_attention()
        self.Fusion_block_0 = Fusion_attention(64, 64 , 54)
        
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.mutex_block_1 = Mutex_attention()
        self.Fusion_block_1 = Fusion_attention(256, 256, 54)
        
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.mutex_block_2 = Mutex_attention()
        self.Fusion_block_2 = Fusion_attention(512, 512, 27)
        
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.mutex_block_3 = Mutex_attention()
        self.Fusion_block_3 = Fusion_attention(1024, 1024, 14)
        
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.mutex_block_4 = Mutex_attention()
        self.Fusion_block_4 = Fusion_attention(2048, 2048, 7)
        
        # pooling
        self.avgpool = nn.AvgPool2d(7)
        
        self.fc_o_1 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(2048, 512),
            nn.ReLU())
        self.fc_m_1 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(2048, 512),
            nn.ReLU())
        
        self.fc_o_2 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(512, 2),
            nn.ReLU())
        self.fc_m_2 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(512, 2),
            nn.ReLU())
        
        self.softmax_out = nn.Softmax(dim=1)
        
        # self.linear = nn.Linear(512*block.expansion, num_classes)
        # self.linear = nn.Linear(100352, num_classes)


    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)


    def forward(self, x, mutex_x):
        
        
        # Res_layer_0
        # print('Res_layer_0')
        Frei = F.relu(self.layer0(x))
        Frei = nn.MaxPool2d(kernel_size = 3, stride = 2)(Frei)
        
        Frem = F.relu(self.layer0(mutex_x))
        Frem = nn.MaxPool2d(kernel_size = 3, stride = 2)(Frem)
        
        Fam = self.mutex_block_0(Frei, Frem)
        
        Fom = self.Fusion_block_0(Fam, Frem)
        
        Fo = Frei
        
        # Set names
        Fi = Fo
        Fm = Fom
        # print(f'shape of Fo is: {Fo.shape}')
        # print(50*'-')
        
        # ---------------------------------------------------------
        
        # Res_layer_1
        # print('Res_layer_1')
        
        Frei = F.relu(self.layer1(Fi))
        
        Frem = F.relu(self.layer1(Fm))
        
        Fam = self.mutex_block_1(Frei, Frem)
        
        Fom = self.Fusion_block_1(Fam, Frem)
        
        Fo = Frei
        
        # Set names
        Fi = Fo
        Fm = Fom
        # print(f'shape of Fo is: {Fo.shape}')
        # print(50*'-')
        
        # ---------------------------------------------------------
        
        # Res_layer_2
        # print('Res_layer_2')
        
        Frei = F.relu(self.layer2(Fi))
        
        Frem = F.relu(self.layer2(Fm))
        
        Fam = self.mutex_block_2(Frei, Frem)
        
        Fom = self.Fusion_block_2(Fam, Frem)
        
        Fo = Frei
        
        # Set names
        Fi = Fo
        Fm = Fom
        # print(f'shape of Fo is: {Fo.shape}')
        # print(50*'-')
        
        # ---------------------------------------------------------
        
        # Res_layer_3
        # print('Res_layer_3')
        
        Frei = F.relu(self.layer3(Fi))
        
        Frem = F.relu(self.layer3(Fm))
        
        Fam = self.mutex_block_3(Frei, Frem)
        
        Fom = self.Fusion_block_3(Fam, Frem)
        
        Fo = Frei
        
        # Set names
        Fi = Fo
        Fm = Fom
        # print(f'shape of Fo is: {Fo.shape}')
        # print(50*'-')
        
        # ---------------------------------------------------------
        
        # Res_layer_4
        # print('Res_layer_4')
        
        Frei = F.relu(self.layer4(Fi))
        
        Frem = F.relu(self.layer4(Fm))
        
        Fam = self.mutex_block_4(Frei, Frem)
        
        Fom = self.Fusion_block_4(Fam, Frem)
        
        Fo = Frei
        
        # Set names
        Fi = Fo
        Fm = Fom
        
        #set Vi, Vm for calculate cosine loss
        Vi = Fo.clone()
        Vm = Fom.clone()
        # print(f'shape of Fo is: {Fo.shape}')
        # print(50*'-')
        
        # ---------------------------------------------------------
        
        # global pooling
        Fo = self.avgpool(Fo).squeeze()
        Fom = self.avgpool(Fom).squeeze()
        # print(f'shape of Fo , Fm after pooling:\n \
        #         {Fo.shape}, {Fom.shape}')
        
        # ---------------------------------------------------------
        
        # final_fully_layer
        Fo = self.fc_o_1(Fo)
        Fom = self.fc_m_1(Fom)
        
        Fo = self.fc_o_2(Fo)
        Fom = self.fc_m_2(Fom)
        
        Fo = self.softmax_out(Fo)
        Fom = self.softmax_out(Fom)
        
        # print(f'Fo finel:{Fo.shape}')
        # print(f'Fm finel:{Fom.shape}')
        
        # out = self.layer1(out)
        # out = self.layer2(out)
        # out = self.layer3(out)
        # out = self.layer4(out)
        # out = F.avg_pool2d(out, 4)
        # out = out.view(out.size(0), -1)
        # # print(out.shape)
        # out = self.linear(out)
        return Fo, Fom, Vi, Vm

In [112]:
model = ResNet(Bottleneck, [3, 4, 6, 3], Mutex_block, Fusion_block)

In [113]:
model = model.to(device)

# Training

In [114]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.005, weight_decay = 0.005, momentum = 0.9)

In [115]:
# initial losses
Lce_ = nn.CrossEntropyLoss()
Lcs_ = nn.CosineSimilarity(dim=1, eps=1e-6)

In [116]:
for sample in train_loader:
  image_i = sample['image_i'].to(device)
  label_i = sample['label_i'].to(device)
  image_mu = sample['image_mu'].to(device)
  label_mu = sample['label_mu'].to(device)

  Fo, Fom, Vi, Vm = model(image_i.float(), image_mu.float())

  # calculate losses
  L1 = Lce_(Fo, label_i)
  L2 = Lce_(Fom, label_mu)
  Lce = L1 + L2

  Lcs = torch.mean(Lcs_(Vi.view(Vi.size(0), -1), Vm.view(Vm.size(0), -1)))

  Lce_exp = torch.exp(1/Lce)
  Lcs_exp = torch.exp(1/Lcs)
  a1 = Lce_exp / (Lce_exp + Lcs_exp)
  a2 = Lcs_exp / (Lce_exp + Lcs_exp)

  loss = (a1 * Lce) + (a2 * Lcs)

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  
  print(loss)

tensor(0.6120, grad_fn=<AddBackward0>)
tensor(0.6430, grad_fn=<AddBackward0>)
tensor(0.6318, grad_fn=<AddBackward0>)
tensor(0.6322, grad_fn=<AddBackward0>)
tensor(0.6179, grad_fn=<AddBackward0>)
tensor(0.6174, grad_fn=<AddBackward0>)
tensor(0.6098, grad_fn=<AddBackward0>)
tensor(0.6207, grad_fn=<AddBackward0>)
tensor(0.6059, grad_fn=<AddBackward0>)
tensor(0.5900, grad_fn=<AddBackward0>)
tensor(0.6015, grad_fn=<AddBackward0>)
tensor(0.5867, grad_fn=<AddBackward0>)
tensor(0.5704, grad_fn=<AddBackward0>)
tensor(0.5263, grad_fn=<AddBackward0>)
tensor(0.5703, grad_fn=<AddBackward0>)
tensor(0.5714, grad_fn=<AddBackward0>)
tensor(0.5482, grad_fn=<AddBackward0>)
tensor(0.5463, grad_fn=<AddBackward0>)
tensor(0.5245, grad_fn=<AddBackward0>)
tensor(0.5081, grad_fn=<AddBackward0>)
tensor(0.6010, grad_fn=<AddBackward0>)
tensor(0.5192, grad_fn=<AddBackward0>)
tensor(0.5557, grad_fn=<AddBackward0>)
tensor(0.5030, grad_fn=<AddBackward0>)
tensor(0.5460, grad_fn=<AddBackward0>)
tensor(0.5555, grad_fn=<A

In [107]:
loss

tensor(0.6289, grad_fn=<AddBackward0>)